In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
proj_dir = Path.cwd().parent.parent.parent


In [10]:
# load compiled_results_filtered.csv
df = pd.read_csv('compiled_results_filtered.csv')
df.head()

,uo_method,scaler_method,imbalance_ratio,classifier,sampler_seed,feat_col_list,date_time,dataset,precision_score_min,precision_score_max,...,SGDClassifier_loss,SGDClassifier_penalty,RandomForestClassifier_bootstrap,RandomForestClassifier_class_weight,RandomForestClassifier_criterion,RandomForestClassifier_max_depth,RandomForestClassifier_min_samples_leaf,RandomForestClassifier_min_samples_split,RandomForestClassifier_n_estimators,SVC_degree
0,random_over,minmax,0.5,rf,6325413,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.920912,0.968493,...,NaN,NaN,False,NaN,entropy,58.0,7.0,5.0,382.0,NaN
1,smote,minmax,1.0,rf,16934992,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.743258,0.836218,...,NaN,NaN,False,balanced_subsample,gini,49.0,5.0,8.0,309.0,NaN
2,adasyn,minmax,1.0,rf,17610750,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.710010,0.807792,...,NaN,NaN,False,balanced_subsample,gini,476.0,5.0,8.0,15.0,NaN
3,smote,minmax,0.8,rf,25570997,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.662149,0.727804,...,NaN,NaN,False,balanced_subsample,entropy,406.0,6.0,7.0,39.0,NaN
4,adasyn,minmax,0.3,rf,8799181,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.929308,0.976705,...,NaN,NaN,True,NaN,gini,494.0,2.0,7.0,173.0,NaN


In [7]:
mill_feature_path = proj_dir / "data/processed/milling/milling_features.csv.gz"
df_feat = pd.read_csv(mill_feature_path, compression='gzip')
df_feat.head()

,cut_id,ae_spindle__length,ae_spindle__kurtosis,ae_table__length,ae_table__kurtosis,vib_spindle__length,vib_spindle__kurtosis,vib_table__length,vib_table__kurtosis,smcdc__length,smcdc__kurtosis,smcac__length,smcac__kurtosis,cut_no,case,tool_class
0,50_23,64.0,-0.536421,64.0,1.023564,64.0,0.052618,64.0,-1.182274,64.0,-0.741543,64.0,-1.379993,50,4,1
1,50_24,64.0,0.580646,64.0,-0.248433,64.0,-0.477432,64.0,-1.185010,64.0,-0.906486,64.0,-1.373383,50,4,1
2,50_25,64.0,-0.335575,64.0,-0.117931,64.0,0.385026,64.0,-1.320485,64.0,-1.051426,64.0,-1.398157,50,4,1
3,50_26,64.0,-1.180998,64.0,-0.227941,64.0,0.198078,64.0,-1.123297,64.0,-0.680110,64.0,-1.348922,50,4,1
4,50_27,64.0,-0.074406,64.0,-0.431231,64.0,-0.430192,64.0,-1.300146,64.0,-0.387458,64.0,-1.363613,50,4,1


In [18]:
from src.models.random_search_setup import (
    rf_params,
    xgb_params,
    knn_params,
    lr_params,
    sgd_params,
    ridge_params,
    svm_params,
    nb_params,
    general_params
)

general_params.keys()

dict_keys(['scaler_method', 'uo_method', 'imbalance_ratio', 'classifier'])

In [20]:
general_params_single = {}
for k in general_params.keys():
    general_params_single[k] = [df.iloc[0][k]]

In [23]:
# make a dict comprehension for general_params_single
general_params_single = {k: [df.iloc[0][k]] for k in general_params.keys()}
general_params_single

{'scaler_method': ['minmax'],
 'uo_method': ['random_over'],
 'imbalance_ratio': [0.5],
 'classifier': ['rf']}

In [21]:
general_params_single

{'scaler_method': ['minmax'],
 'uo_method': ['random_over'],
 'imbalance_ratio': [0.5],
 'classifier': ['rf']}

In [19]:
df.iloc[0]

uo_method                                   random_over
scaler_method                                    minmax
imbalance_ratio                                     0.5
classifier                                           rf
sampler_seed                                    6325413
                                               ...     
RandomForestClassifier_max_depth                   58.0
RandomForestClassifier_min_samples_leaf             7.0
RandomForestClassifier_min_samples_split            5.0
RandomForestClassifier_n_estimators               382.0
SVC_degree                                          NaN
Name: 0, Length: 69, dtype: object